In [1]:
from contextlib import contextmanager
from typing import List, Union
import sys
import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import rioxarray as rxr
from sklearn.metrics import accuracy_score,recall_score
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import (f1_score, accuracy_score, precision_score, recall_score, 
                             confusion_matrix, matthews_corrcoef, cohen_kappa_score,
                             hamming_loss, classification_report)
#from minisom import MiniSom
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import rasterio
from rasterio.windows import Window

os.environ["DEVICE"] = "cuda"
@contextmanager
def cwd(path: str) -> None:

    """
    Context manager para mudar o diretório de trabalho.
    Mantém o diretório original após a execução do bloc
    
    o de código.
    """

    oldpwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)
np.set_printoptions(threshold=sys.maxsize, edgeitems=sys.maxsize, linewidth=sys.maxsize, precision=sys.maxsize)

In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import numpy as np

from s2cloudless import S2PixelCloudDetector, download_bands_and_valid_data_mask

import datetime as dt
from datetime import datetime

import rasterio
from osgeo import gdal
import osgeo_utils.gdal_merge

from numba import jit
from functools import lru_cache

import subprocess
import os
import glob
import json
import shutil

from typing import List, Tuple
from contextlib import contextmanager

In [ ]:
@contextmanager
def cwd(path: str) -> None:
    
    """
    Context manager para mudar o diretório de trabalho.
    Mantém o diretório original após a execução do bloco de código.
    """
    
    oldpwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)

def listdir_fullpath(d: str) -> List[str]:
    
    """
    Retorna uma lista de caminhos completos para os arquivos em um diretório.
    """
    
    return [os.path.join(d, f) for f in sorted(os.listdir(d))]

class AmbienteIncorretoError(Exception):
    def __init__(self):
        self.message = "Necessário estar no ambiente 'km_predict' para rodar o modelo"
        super().__init__(self.message)


class Preprocessing:
    def __init__(self, caminho: str) -> None:
        self.caminho: str = caminho  # Pasta onde as imagens .SAFE estão
        self.abs_caminho: str = os.path.abspath(self.caminho)
        self.imgs_diretorio: List[str] = os.listdir(self.caminho)  # Nome de cada imagem .SAFE no diretório
        self.diretorios_tif: List[str] = sorted([diretorio.replace('.SAFE', '.TIF') for diretorio in self.imgs_diretorio if diretorio.endswith('.SAFE')])
        self.caminho_completo_lista: List[str] = [item for item in listdir_fullpath(self.caminho) if item.endswith('.SAFE')]

    def merge_tif_files(self, nome_TIF: str, output_name:str) -> None:
        """
        Mescla arquivos TIFF em um único arquivo usando gdal_merge.
        """
        with cwd(nome_TIF):
            if os.path.exists(output_name):
                os.remove(output_name)
            arquivos_tif: List[str] = glob.glob('*B*.tif')
            arquivos_tif: List[str] = self._sort_files(arquivos_tif,sufix='.tif')
            comando: List[str] = ['gdal_merge.py', '-o', output_name, '-of', 'Gtiff', '-separate', '-ot', 'FLOAT32', '-co', 'BIGTIFF=YES'] + arquivos_tif
            parameters = ['', '-o', output_name] + arquivos_tif + ['-separate', '-co', 'COMPRESS=LZW','-co','BIGTIFF=YES','-co', 'COMPRESS=LZW']
            osgeo_utils.gdal_merge.main(parameters)
            list(map(os.remove, arquivos_tif))

    def _create_tif_folder(self) -> None:
        """
        Cria pastas que serão utilizadas para o resultado dos modelos.
        """
        for tif_dir in self.diretorios_tif:
            if os.path.exists(tif_dir):
                shutil.rmtree(f'./{tif_dir}')
            os.makedirs(tif_dir)
        self._modify_img_diretorio()

    def _modify_img_diretorio(self) -> None:
        self.imgs_diretorio: List[str] = [diretorio for diretorio in self.imgs_diretorio if diretorio.endswith('.SAFE')]

    def jp2_to_tif(self, tif_dim: List[str] = ['10980','10980'], output_name: str = 'merge.tif', create_folder:bool = True) -> None:
        """
        Converte arquivos JP2 para TIFF usando gdal_translate.
        """
        with cwd(self.caminho):
            if create_folder:
                self._create_tif_folder()
            for diretorio in self.imgs_diretorio:
                files: List[str] = glob.glob(os.path.join(f'{diretorio}', 'GRANULE', '*', 'IMG_DATA', '*B*.jp2'))
                files: List[str] = self._sort_files(files,sufix='.jp2')
                nome_TIF: str = diretorio.replace('.SAFE', '.TIF')
                commands: List[List[str]] = []
                
                for f in files:
                    input_path: str = f
                    output_path: str = nome_TIF + '/' + os.path.splitext(os.path.basename(f))[0] + '.tif'
                    if os.path.exists(output_path):
                        os.remove(output_path)
                    cmd: List[str] = ['gdal_translate', input_path, '-ot', 'Float32', '-of', 'Gtiff', '-outsize', tif_dim[0], tif_dim[1], output_path, '-co', 'BIGTIFF=YES']
                    commands.append(cmd)
                    
                for cmd in commands:
                    subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    
                self.merge_tif_files(nome_TIF,output_name=output_name)
            
                    
                    
    def _sort_files(self, lista: list, sufix: str):
        lista = sorted(lista)

        b08_name: int = [item for item in lista if item.endswith(f"B08{sufix}")][0]
        b08_index: int = lista.index(b08_name)

        b8A_name: int = [item for item in lista if item.endswith(f"B8A{sufix}")][0]
        b8A_index: int = lista.index(b8A_name)

        lista.insert(b08_index + 1, lista.pop(b8A_index))
        
        return lista

    def _criar_symlinks_kappaMask(self, kappa_mask_folder: str) -> None:
        """
        Cria symlinks para pasta "data" do kappamask.
        """
        with cwd(kappa_mask_folder):
            if not os.path.exists('./data'):
                os.makedirs('data')
                
        pasta_origem: str = os.path.abspath(kappa_mask_folder)
        for diretorio in self.caminho_completo_lista:
            caminho_pasta: str = self.caminho + '/' + diretorio
            pasta_destino: str = os.path.abspath(caminho_pasta)
            symlink_path: str = os.path.join(pasta_origem, "data", os.path.basename(pasta_destino))
            try:
                os.symlink(pasta_destino, symlink_path)
                print(f"Symlink criado com sucesso: {symlink_path} -> {pasta_destino}")
            except FileExistsError:
                return
            except Exception as e:
                print(f"Erro ao criar symlink: {e}")

    def _config_json_kappa(self, kappa_mask_folder, product_name, architecture, json_config_name) -> None:
        diretorio_atual = self.abs_caminho
        print(diretorio_atual)
        json_config: dict = {
            "cm_vsm_executable": "cm_vsm",
            "folder_name": f"{diretorio_atual}",
            "product_name": f"{product_name.replace('.SAFE','')}",
            "level_product": "L1C",
            "overlapping": 0.0625,
            "tile_size": 512,
            "resampling_method": "sinc",
            "architecture": f"{architecture}",
            "batch_size": 1,
            "aoi_geometry": ""
        }
        caminho_json_config: str = f'./config/{json_config_name}'
        with cwd(kappa_mask_folder):
            with open(caminho_json_config, 'w') as arquivo:
                json.dump(json_config, arquivo, indent=2)

class Modelos(Preprocessing):
    def __init__(self, caminho: str) -> None:
        self.caminho = caminho
        super().__init__(caminho)

    def start(self,create_folder) -> None:
        """Começa a etapa de pré-processamento dos arquivos .jp2"""
        self.jp2_to_tif(create_folder=create_folder)
        df = Modelos.get_gt_predict(self.caminho.split('/')[-1].upper())
        with cwd(self.caminho):
            list(map(shutil.rmtree, self.diretorios_tif))
        return df
        
    @staticmethod
    def read_tif_values(tif_path, x, y):
        with rasterio.open(tif_path) as src:
            # Converte as coordenadas x, y em índices de linha e coluna no raster
            row, col = src.index(x, y)
            values = []
            for i in range(1, src.count + 1):  # Para cada banda
                window = Window(col, row, 1, 1)
                band_values = src.read(i, window=window)
                values.append(band_values[0][0])
            return values

    @staticmethod
    def tifs(df_rotulos, index, name,regiao):
        ponto_de_montagem = f'/media/jean/90D8B801D8B7E41E/Ubuntu/{regiao.title()}'
        destino = os.path.abspath(ponto_de_montagem)
        with cwd(destino):
            with cwd(name):
                geotiff_path = 'merge.tif'
                # Lista para armazenar os valores das bandas
                band_values_list = []

                # Itera sobre cada linha do dataframe
                for _, row in df_rotulos.iterrows():
                    x, y = row['x'], row['y']
                    values = Modelos.read_tif_values(geotiff_path, x, y)
                    band_values_list.append(values)

                # Adiciona os valores das bandas ao dataframe original
                band_values_array = np.array(band_values_list)
                for i in range(band_values_array.shape[1]):
                    df_rotulos[f'band_{i + 1}'] = band_values_array[:, i]

                return df_rotulos
            
    @staticmethod
    def get_gt_predict(regiao):
        def get_value_from_data_df(row):
            return data_df[row['x']][float(row['y'])]
        resultado = {}
        with cwd('./Pontos_Validados/'):
            with cwd(f'./{regiao.upper()}'):
                dates = [x for x in os.listdir() if not x in (".ipynb_checkpoints")]
                i=0
                for data in dates:
                    date = f'{data}/'
                    print(date)
                    with cwd(date):
                        models = [x for x in sorted(os.listdir()) if ( x != ".ipynb_checkpoints") and (not(x.endswith('.xml'))) and (not(x.endswith('.jp2')))]
                        for model in models:
                            with cwd(os.path.join(model)):
                                xml = [x for x in os.listdir(f'../') if x.endswith("xml")][0].split('_')[4:11]
                                name = '_'.join(xml).replace('.SAFE','.TIF')
                                files = sorted(os.listdir())  
                                shp = [shp for shp in files if shp.endswith('.shp')][0]
                                shapefile = gpd.read_file(shp)
                                new_df = shapefile['geometry'].get_coordinates()
                                df = pd.concat([new_df,shapefile['GrndTruth']],axis=1)
                                df['datetime'] = pd.to_datetime(data.replace('(1)','').replace('_','/'),dayfirst=True)
                                df = Modelos.tifs(df,i,name,regiao)
                                resultado[f'{regiao}_{model}_{data}'] = df
        return resultado

pasta_principal = '/media/jean/90D8B801D8B7E41E/Ubuntu/'
for i in os.listdir(pasta_principal)[1:]:
    pasta_regiao = os.path.abspath(os.path.join(pasta_principal,i))
    obj: Modelos = Modelos(pasta_regiao)
    teste_df_regiao = obj.start(create_folder=True)  # Executar uma ÚNICA vez na pasta no qual as imagens estão.
    pd.concat(teste_df_regiao.values(),axis=0).to_csv(f'{i.title()}_pixels.csv')

...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60

In [10]:
for i in os.listdir('/media/jean/90D8B801D8B7E41E/Ubuntu/')[1:]:
    print("/media/jean/90D8B801D8B7E41E/Ubuntu/",i)

/media/jean/90D8B801D8B7E41E/Ubuntu/ Hiwi
/media/jean/90D8B801D8B7E41E/Ubuntu/ Ipoá
/media/jean/90D8B801D8B7E41E/Ubuntu/ Itaúba
/media/jean/90D8B801D8B7E41E/Ubuntu/ Ybyra


In [2]:

teste = get_gt_predict('Awa')
#df = get_gt_predict('Hiwi')
#df_rotulos = pd.concat(df_rotulos,axis=0)
#df_rotulos.to_csv('YBYRA.csv')

In [ ]:
for _, row in teste.iterrows():
    x, y = row['x'], row['y']
    values = read_tif_values(geotiff_path, x, y)
    band_values_list.append(values

In [3]:
teste

,x,y,GrndTruth,datetime
0,402210.0,9898150.0,4,2023-11-29
1,384730.0,9897110.0,0,2023-11-29
2,393050.0,9895110.0,4,2023-11-29
3,393490.0,9895010.0,4,2023-11-29
4,399250.0,9893990.0,2,2023-11-29
...,...,...,...,...
244,401030.0,9792530.0,4,2023-11-29
245,401110.0,9791050.0,4,2023-11-29
246,389590.0,9790490.0,4,2023-11-29
247,403010.0,9795490.0,4,2023-11-29


In [72]:
df_pixels_bandas = pd.concat(list(df.values()),axis=0)
df_pixels_bandas.to_csv('HIWI_pixels.csv')

In [76]:
pd.read_csv('./AWA_pixels.csv')

,Unnamed: 0,x,y,GrndTruth,datetime,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,band_11,band_12,band_13
0,0,402210.0,9898150.0,4,2023-11-29,7028.0,6723.0,6521.0,6903.0,7193.0,7710.0,8058.0,7140.0,8309.0,2823.0,1018.0,6397.0,4917.0
1,1,384730.0,9897110.0,0,2023-11-29,2774.0,2347.0,2210.0,1745.0,2239.0,4396.0,5434.0,5210.0,6086.0,1417.0,1011.0,3103.0,1732.0
2,2,393050.0,9895110.0,4,2023-11-29,9975.0,10063.0,9885.0,10596.0,10356.0,10755.0,11248.0,10239.0,11473.0,3641.0,1020.0,7315.0,5150.0
3,3,393490.0,9895010.0,4,2023-11-29,9850.0,9700.0,9775.0,10781.0,10439.0,11107.0,11904.0,10216.0,12351.0,3040.0,1019.0,7428.0,5331.0
4,4,399250.0,9893990.0,2,2023-11-29,2630.0,2219.0,2000.0,1609.0,1817.0,2819.0,3260.0,3243.0,3556.0,1226.0,1009.0,1917.0,1294.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,243,342370.0,9797050.0,9,2023-10-28,6016.0,5875.0,5936.0,6472.0,6268.0,6687.0,7117.0,6656.0,7436.0,2385.0,1019.0,6787.0,5561.0
3718,244,358350.0,9795890.0,9,2023-10-28,4998.0,5076.0,4908.0,4989.0,4891.0,5410.0,5818.0,5629.0,6077.0,1962.0,1020.0,5602.0,4422.0
3719,245,346770.0,9791090.0,8,2023-10-28,3647.0,3130.0,3065.0,2991.0,3028.0,3987.0,4695.0,4397.0,5019.0,1518.0,1016.0,3274.0,2344.0
3720,246,362990.0,9797270.0,9,2023-10-28,7498.0,7366.0,7254.0,7674.0,7464.0,7625.0,7988.0,7568.0,8209.0,2726.0,1024.0,6477.0,4832.0


In [75]:
df_pixels_bandas

,x,y,GrndTruth,datetime,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,band_11,band_12,band_13
0,611090.000000,8.999950e+06,2,2023-11-27,2476.0,2383.0,2144.0,1812.0,2064.0,3091.0,3672.0,3606.0,3957.0,1429.0,1040.0,2425.0,1773.0
1,602570.000000,8.995970e+06,4,2023-11-27,3498.0,3255.0,3098.0,2931.0,3136.0,4304.0,4993.0,4685.0,5398.0,1914.0,1051.0,3500.0,2604.0
2,619850.000000,8.992930e+06,4,2023-11-27,4425.0,3909.0,3899.0,3902.0,4082.0,4840.0,5360.0,4936.0,5617.0,2937.0,1160.0,4057.0,3256.0
3,603090.000000,8.992390e+06,0,2023-11-27,2208.0,1908.0,1804.0,1438.0,1775.0,3097.0,3679.0,3545.0,4119.0,1282.0,1013.0,2202.0,1408.0
4,616550.000000,8.990090e+06,4,2023-11-27,3710.0,3577.0,3398.0,3228.0,3389.0,4259.0,4784.0,4629.0,5088.0,2172.0,1095.0,3715.0,2931.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,656761.161253,8.955278e+06,4,2023-11-27,2202.0,2061.0,1997.0,1703.0,1982.0,3627.0,4462.0,4010.0,5007.0,1331.0,1015.0,2652.0,1598.0
232,631653.191382,8.974185e+06,9,2023-11-27,5656.0,5384.0,5298.0,5386.0,5568.0,6283.0,6729.0,6406.0,6969.0,3226.0,1051.0,4433.0,2904.0
233,612696.459681,8.926653e+06,9,2023-11-27,2784.0,3084.0,2890.0,2438.0,2603.0,3640.0,4248.0,4471.0,4666.0,1438.0,1094.0,3164.0,2039.0
234,694197.702507,8.975524e+06,9,2023-11-27,4833.0,4078.0,4331.0,4403.0,4571.0,5312.0,5751.0,5232.0,6022.0,3433.0,1234.0,4571.0,3673.0


In [3]:
def create_df():
    df_todos = []
    for i in [csv for csv in sorted(os.listdir()) if csv.endswith('.csv')]:
        df_todos.append(pd.read_csv(i))
    return pd.concat(df_todos,axis=0)
df_todos = create_df()
df_todos = df_todos.reset_index().drop(['index','Unnamed: 0','band_14','band_15','band_16','row','col'],axis=1)

X = df_todos.drop(['x','y','datetime','value','GrndTruth'],axis=1)
y = df_todos['GrndTruth']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [12]:
pd.concat([X_test.reset_index().drop('index',axis=1),pd.Series(y_testando)],axis=1)

,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,band_11,band_12,band_13,0
0,6283.0,6787.0,6396.0,6473.0,6179.0,6483.0,6953.0,7006.0,7102.0,2343.0,1019.0,5680.0,4282.0,4
1,2367.0,2134.0,2088.0,2115.0,2224.0,2385.0,2635.0,2318.0,2522.0,1189.0,1011.0,1790.0,1431.0,1
2,2209.0,1930.0,1791.0,1521.0,1765.0,3074.0,3758.0,3760.0,4261.0,1417.0,1011.0,2551.0,1603.0,0
3,2408.0,2060.0,1942.0,1594.0,2010.0,3371.0,3978.0,3842.0,4437.0,1401.0,1009.0,2630.0,1594.0,0
4,2265.0,2095.0,2083.0,2219.0,2471.0,2964.0,3357.0,3331.0,3830.0,1461.0,1050.0,4203.0,2807.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,2449.0,2071.0,1889.0,1650.0,1800.0,2346.0,2644.0,2354.0,2774.0,1275.0,1029.0,2006.0,1428.0,2
1214,6212.0,6000.0,5745.0,5981.0,6034.0,6465.0,6981.0,6644.0,7255.0,2592.0,1022.0,5988.0,4796.0,4
1215,6946.0,6710.0,6609.0,6970.0,6993.0,7735.0,8305.0,7480.0,8839.0,2191.0,1020.0,7507.0,5875.0,4
1216,2643.0,2201.0,2076.0,1667.0,2080.0,3695.0,4627.0,4593.0,5030.0,1538.0,1010.0,2829.0,1696.0,0


In [13]:
with cwd('YBYRA/Imagens'):
    with cwd("S2B_MSIL1C_20231128T133139_N0509_R081_T22MHB_20231128T151020.TIF"):
        geotiff = rxr.open_rasterio(f'merge.tif')

        # Ler cada banda e armazenar os dados na lista
        bands_data = []
        for band_path in range(geotiff.rio.count):
            band_data = geotiff[band_path].values
            bands_data.append(band_data.flatten())  # Achata a matriz e adiciona à lista

        height, width = band_data.shape
        rows, cols = np.indices((height, width))
        rows = rows.flatten()
        cols = cols.flatten()

        # Calcula as coordenadas X e Y usando a transformação geoespacial
        x_coords = geotiff.x.values
        y_coords = geotiff.y.values
        xs, ys = np.meshgrid(x_coords, y_coords)
        xs = xs.flatten()
        ys = ys.flatten()

        # Cria um dicionário com as colunas row, col, x, y e cada banda
        data = {
            'row': rows,
            'col': cols,
            'x': xs,
            'y': ys
        }

        # Adiciona cada banda ao dicionário
        for i, band_flat in enumerate(bands_data):
            data[f'band_{i+1}'] = band_flat

        # Criação do DataFrame
        df = pd.DataFrame(data)
        del data

In [ ]:
df.drop(['row','col','x','y'],axis=1).values.shape

In [ ]:
# # Defina uma função de métrica personalizada
# def custom_metrics(y_true, y_pred):
#     metrics = {}
#     metrics['accuracy'] = accuracy_score(y_true, y_pred)
#     metrics['precision_weighted'] = precision_score(y_true, y_pred, average='weighted')
#     metrics['recall_weighted'] = recall_score(y_true, y_pred, average='weighted')
#     metrics['f1_weighted'] = f1_score(y_true, y_pred, average='weighted')
#     metrics['cohen_kappa'] = cohen_kappa_score(y_true, y_pred)
    
#     return metrics

# # Crie e ajuste o LazyClassifier
# clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=custom_metrics)
# models, predictions = clf.fit(X_train, X_test, y_train, y_test)

In [ ]:
# def construct_series(model,y_true,y_pred):
#     serie = pd.Series(custom_metrics(y_true,y_pred)).to_frame().T
#     serie['Model'] = model
#     serie.index = serie['Model']
#     serie = serie.drop('Model',axis=1)
#     return serie


# dfs_metricas = []
# for value in predictions['custom_metrics'].values:
#     serie = pd.Series(value).to_frame().T
#     serie['Model'] = predictions.index[list(predictions['custom_metrics'].values).index(value)]
#     serie.index = serie['Model']
#     serie = serie.drop('Model',axis=1)
#     dfs_metricas.append(serie)
    
# fmask_results = construct_series('Fmask',df_todos['GrndTruth'],df_todos['value'])
# dfs_metricas = pd.concat(dfs_metricas,axis=0)
# dfs_metricas = pd.concat([dfs_metricas,fmask_results],axis=0).sort_values(by='accuracy',ascending=False).reset_index()

In [ ]:
# from tpot import TPOTClassifier

# pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,random_state=42, verbosity=2)
# pipeline_optimizer.fit(X_train, y_train)
# y_pred = pipeline_optimizer.predict(X_test)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, accuracy_score

# # Supondo y_true e y_pred como os rótulos verdadeiros e as previsões do modelo, respectivamente
# y_true = y_test

# # Matriz de Confusão
# conf_matrix = confusion_matrix(y_true, y_pred)
# plt.figure(figsize=(10, 8))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.title('Confusion Matrix')
# plt.show()

# # Relatório de Classificação
# report = classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()

# plt.figure(figsize=(10, 8))
# sns.heatmap(report_df.iloc[:-1, :].T, annot=True, cmap='Blues')
# plt.title('Classification Report')
# plt.show()

# # Gráfico de Barras para Métricas de Cada Classe
# precision = report_df['precision'][:-1]
# recall = report_df['recall'][:-1]
# f1 = report_df['f1-score'][:-1]

# metrics_df = pd.DataFrame({'precision': precision, 'recall': recall, 'f1-score': f1})
# metrics_df.plot(kind='bar', figsize=(12, 8))
# plt.title('Precision, Recall, and F1-Score for Each Class')
# plt.xlabel('Classes')
# plt.ylabel('Score')
# plt.xticks(rotation=45)
# plt.ylim(0, 1)
# plt.legend(loc='lower right')
# plt.show()

# # Gráfico de Erro
# errors = (y_true != y_pred)
# error_classes = pd.Series(y_true[errors]).value_counts()

# plt.figure(figsize=(12, 8))
# error_classes.plot(kind='bar', color='red')
# plt.title('Number of Errors per Class')
# plt.xlabel('Classes')
# plt.ylabel('Number of Errors')
# plt.show()

# # Matriz de Confusão Normalizada
# conf_matrix_norm = confusion_matrix(y_true, y_pred, normalize='true')
# plt.figure(figsize=(10, 8))
# sns.heatmap(conf_matrix_norm, annot=True, fmt='.2f', cmap='Blues')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.title('Normalized Confusion Matrix')
# plt.show()

# # Outras métricas
# accuracy = accuracy_score(y_true, y_pred)
# balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
# kappa = cohen_kappa_score(y_true, y_pred)

# print(f"Acurácia: {accuracy}")
# print(f"Acurácia Balanceada: {balanced_accuracy}")
# print(f"Kappa Estatístico: {kappa}")